
# ✅ Global LSTM (10‑Day Direction) with Per‑Ticker Calibrators

This notebook trains a **single global LSTM** across a mixed universe (stocks/ETFs/crypto) to predict **10‑day direction (up/down)**, **saves** the model + scaler, and then fits **tiny per‑ticker logistic calibrators** (Platt scaling) on the most recent N months to convert the global score/logit into a **ticker‑specific probability**.

### What you'll get
- `models/global/lstm_model.keras` – the **global** Keras model
- `models/global/scaler.pkl` – `StandardScaler` used to scale features
- `models/calibrators/<TICKER>.json` – `{A,B}` params for each ticker's calibration
- A test cell that outputs a **forecast payload** compatible with your current API/FE shape

> You can run this nightly (or convert to a Python script later). The frontend **does not need to change**—you keep returning the same `forecast` list.


## 0) Setup & Install

In [3]:

# --- If needed in a fresh environment, UNCOMMENT the installs below ---
# Linux/Windows (CPU):
# !pip install tensorflow==2.16.1
#
# Apple Silicon (M1/M2/M3):
# !pip install tensorflow-macos==2.16.1 tensorflow-metal==1.1.0
#
# Common deps:
# !pip install yfinance==0.2.43 pandas==2.2.3 numpy==1.26.4 scikit-learn==1.5.2 joblib==1.4.2 Pillow==10.4.0

import os, json, joblib, io, base64, math, warnings, random
from typing import List, Tuple, Dict
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import yfinance as yf

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from scipy.special import expit as sigmoid

warnings.filterwarnings("ignore")

# Determinism (best-effort)
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# Paths
try:
    # Works if running as a .py script
    NOTEBOOK_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Fallback for Jupyter: get the absolute path of the notebook itself
    import pathlib
    NOTEBOOK_DIR = str(pathlib.Path().resolve())

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)

MODELS_DIR = os.path.join(NOTEBOOK_DIR, "models")
GLOBAL_DIR = os.path.join(MODELS_DIR, "global")
CALIB_DIR  = os.path.join(MODELS_DIR, "calibrators")

for p in [MODELS_DIR, GLOBAL_DIR, CALIB_DIR]:
    os.makedirs(p, exist_ok=True)
    print("ensured:", p, "exists?", os.path.isdir(p))

MODEL_PATH  = os.path.join(GLOBAL_DIR, "lstm_model.keras")
SCALER_PATH = os.path.join(GLOBAL_DIR, "scaler.pkl")
print("MODEL_PATH:", MODEL_PATH)
print("SCALER_PATH:", SCALER_PATH)
# Knobs
LOOK_BACK = 60           # sequence length (days)
HORIZON   = 10           # predict 10-day direction
YEARS     = 5            # history to download
CAL_WINDOW_MONTHS = 9    # window for per-ticker calibration
BATCH = 256
EPOCHS = 30


NOTEBOOK_DIR: /Users/nourf/Projects/stock-predictor/backend/notebooks
ensured: /Users/nourf/Projects/stock-predictor/backend/notebooks/models exists? True
ensured: /Users/nourf/Projects/stock-predictor/backend/notebooks/models/global exists? True
ensured: /Users/nourf/Projects/stock-predictor/backend/notebooks/models/calibrators exists? True
MODEL_PATH: /Users/nourf/Projects/stock-predictor/backend/notebooks/models/global/lstm_model.keras
SCALER_PATH: /Users/nourf/Projects/stock-predictor/backend/notebooks/models/global/scaler.pkl


In [4]:
import os
print("CWD:", os.getcwd())
print("MODEL_PATH:", MODEL_PATH)
print("File exists?", os.path.exists(MODEL_PATH))


CWD: /Users/nourf/Projects/stock-predictor/backend/notebooks
MODEL_PATH: /Users/nourf/Projects/stock-predictor/backend/notebooks/models/global/lstm_model.keras
File exists? True


## 1) Universe (edit as you like)

In [5]:

STOCKS = [
    "AAPL","MSFT","NVDA","AMZN","GOOGL","META","TSLA","BRK-B","JPM","V","MA","UNH","JNJ","WMT","PG","HD",
    "KO","PEP","MRK","PFE","ORCL","CSCO","ADBE","CRM","AMD","INTC","NFLX","COST","NKE","DIS","TSM","ASML",
    "SAP","TM","BA","CAT","GE","HON","UPS","MCD","SBUX","BKNG","ABNB","UBER","SNOW","SHOP","SQ","PYPL"
]
ETFS = [
    "SPY","QQQ","DIA","IWM","EEM","EFA","GLD","SLV","TLT","HYG",
    "XLF","XLK","XLE","XLY","XLI","XLV","XLB","XLRE","XLC","XLU"
]
CRYPTO = [
    "BTC-USD","ETH-USD","SOL-USD","BNB-USD","ADA-USD","XRP-USD","DOGE-USD",
    "AVAX-USD","LTC-USD","MATIC-USD","DOT-USD","TON-USD"
]

UNIVERSE = sorted(list(set(STOCKS + ETFS + CRYPTO)))
print("Universe size:", len(UNIVERSE))
print("Sample:", UNIVERSE[:12])


Universe size: 80
Sample: ['AAPL', 'ABNB', 'ADA-USD', 'ADBE', 'AMD', 'AMZN', 'ASML', 'AVAX-USD', 'BA', 'BKNG', 'BNB-USD', 'BRK-B']


## 2) Data & Features

In [6]:

def fetch_ohlcv(ticker: str, years: int = YEARS) -> pd.DataFrame | None:
    end = datetime.utcnow()
    start = end - timedelta(days=int(365.25 * years))
    try:
        df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    except Exception as e:
        print(f"[WARN] {ticker} download failed: {e}")
        return None
    if df is None or df.empty:
        print(f"[WARN] {ticker}: empty data")
        return None
    df = df[["Close","Volume"]].dropna().copy()
    df["Return1"] = df["Close"].pct_change()
    return df

def build_sequence_features(close_window: np.ndarray) -> np.ndarray:
    # Ensure 1-D
    x = np.asarray(close_window).astype(float).reshape(-1)
    if len(x) < 2:
        # Not enough data to compute returns
        rets = np.zeros_like(x)
    else:
        rets = np.diff(x) / x[:-1]
        rets = np.concatenate([[0.0], rets])  # align length

    # Z-score
    if x.std() == 0:
        z = np.zeros_like(x)
    else:
        z = (x - x.mean()) / (x.std() + 1e-8)

    # Scale raw closes
    x_scaled = x / (x.mean() + 1e-8)

    feats = np.stack([x_scaled, rets, z], axis=1)  # (L, 3)
    return feats



def build_samples(df: pd.DataFrame, look_back: int = LOOK_BACK, horizon: int = HORIZON):
    closes = df["Close"].values.astype(float)
    Xs, ys = [], []
    for i in range(look_back, len(df) - horizon):
        win = closes[i - look_back:i]
        Xs.append(build_sequence_features(win))
        fut_ret = closes[i + horizon] / closes[i] - 1.0
        ys.append(1 if fut_ret > 0 else 0)
    if not Xs:
        return None, None
    X = np.stack(Xs)              # (N, L, F)
    y = np.asarray(ys).astype(int)
    return X, y

def build_pooled(universe: List[str]) -> Tuple[np.ndarray, np.ndarray]:
    X_all, y_all = [], []
    for t in universe:
        df = fetch_ohlcv(t)
        if df is None or len(df) < (LOOK_BACK + HORIZON + 50):
            print(f"[SKIP] {t}: not enough length")
            continue
        X, y = build_samples(df)
        if X is None:
            print(f"[SKIP] {t}: no samples built")
            continue
        X_all.append(X); y_all.append(y)
        print(f"[OK] {t}: samples={len(y)}")
    if not X_all:
        raise RuntimeError("No data built. Check tickers/network.")
    X = np.concatenate(X_all, axis=0)
    y = np.concatenate(y_all, axis=0)
    return X, y


## 3) Model

In [7]:

def make_model(n_features: int) -> keras.Model:
    inp = layers.Input(shape=(LOOK_BACK, n_features))
    x = layers.LSTM(96, return_sequences=False)(inp)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(48, activation="relu")(x)
    out = layers.Dense(1)(x)  # raw logit
    model = keras.Model(inp, out)
    model.compile(optimizer=keras.optimizers.Adam(1e-3),
                  loss=keras.losses.BinaryCrossentropy(from_logits=True))
    return model


## 4) Train with TimeSeriesSplit → Save Best → Refit & Save

In [8]:

# Build pooled dataset (this may take several minutes for ~100 tickers)
X_pool, y_pool = build_pooled(UNIVERSE)
N, L, F = X_pool.shape
print("Pooled:", X_pool.shape, y_pool.shape)

# Flatten for scaling
flat = X_pool.reshape(N*L, F)

# TimeSeriesSplit cross-val to pick best model by val loss
tscv = TimeSeriesSplit(n_splits=5)
best_w, best_val = None, 1e9

for fold, (tr_idx, va_idx) in enumerate(tscv.split(X_pool), start=1):
    # Fit scaler on TRAIN indices only (approximate: use repeated indices to align shapes)
    scaler = StandardScaler().fit(flat[tr_idx.repeat(L), :])
    X_tr = scaler.transform(flat[tr_idx.repeat(L), :]).reshape(len(tr_idx), L, F)
    X_va = scaler.transform(flat[va_idx.repeat(L), :]).reshape(len(va_idx), L, F)
    y_tr, y_va = y_pool[tr_idx], y_pool[va_idx]

    model = make_model(F)
    cbs = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)]
    hist = model.fit(X_tr, y_tr, validation_data=(X_va, y_va),
                     epochs=EPOCHS, batch_size=BATCH, verbose=0, callbacks=cbs)
    v = float(min(hist.history["val_loss"]))
    print(f"[Fold {fold}] val_loss={v:.4f}")
    if v < best_val:
        best_val, best_w = v, model.get_weights()

# Final refit: scaler on first 90% (no leakage), early stop on last 10%
cut = int(0.9 * N)
scaler_all = StandardScaler().fit(flat[:cut*L, :])
X_tr_all = scaler_all.transform(flat[:cut*L, :]).reshape(cut, L, F)
X_va_all = scaler_all.transform(flat[cut*L:, :]).reshape(N - cut, L, F)
y_tr_all = y_pool[:cut]
y_va_all = y_pool[cut:]

final = make_model(F)
if best_w is not None:
    final.set_weights(best_w)
cbs = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)]
final.fit(X_tr_all, y_tr_all, validation_data=(X_va_all, y_va_all),
          epochs=EPOCHS, batch_size=BATCH, verbose=0, callbacks=cbs)

# Save global model + scaler
final.save(MODEL_PATH)
joblib.dump(scaler_all, SCALER_PATH)
print("Saved:", MODEL_PATH, SCALER_PATH)


[OK] AAPL: samples=1185
[OK] ABNB: samples=1111
[OK] ADA-USD: samples=1757
[OK] ADBE: samples=1185
[OK] AMD: samples=1185
[OK] AMZN: samples=1185
[OK] ASML: samples=1185
[OK] AVAX-USD: samples=1730
[OK] BA: samples=1185
[OK] BKNG: samples=1185
[OK] BNB-USD: samples=1757
[OK] BRK-B: samples=1185
[OK] BTC-USD: samples=1757
[OK] CAT: samples=1185
[OK] COST: samples=1185
[OK] CRM: samples=1185
[OK] CSCO: samples=1185
[OK] DIA: samples=1185
[OK] DIS: samples=1185
[OK] DOGE-USD: samples=1757
[OK] DOT-USD: samples=1757
[OK] EEM: samples=1185
[OK] EFA: samples=1185
[OK] ETH-USD: samples=1757
[OK] GE: samples=1185
[OK] GLD: samples=1185
[OK] GOOGL: samples=1185
[OK] HD: samples=1185
[OK] HON: samples=1185
[OK] HYG: samples=1185
[OK] INTC: samples=1185
[OK] IWM: samples=1185
[OK] JNJ: samples=1185
[OK] JPM: samples=1185
[OK] KO: samples=1185
[OK] LTC-USD: samples=1757
[OK] MA: samples=1185
[OK] MATIC-USD: samples=1602
[OK] MCD: samples=1185
[OK] META: samples=1185
[OK] MRK: samples=1185
[OK] MSF


1 Failed download:
['SQ']: YFTzMissingError('possibly delisted; no timezone found')


[WARN] SQ: empty data
[SKIP] SQ: not enough length
[OK] TLT: samples=1185
[OK] TM: samples=1185
[OK] TON-USD: samples=1754
[OK] TSLA: samples=1185
[OK] TSM: samples=1185
[OK] UBER: samples=1185
[OK] UNH: samples=1185
[OK] UPS: samples=1185
[OK] V: samples=1185
[OK] WMT: samples=1185
[OK] XLB: samples=1185
[OK] XLC: samples=1185
[OK] XLE: samples=1185
[OK] XLF: samples=1185
[OK] XLI: samples=1185
[OK] XLK: samples=1185
[OK] XLRE: samples=1185
[OK] XLU: samples=1185
[OK] XLV: samples=1185
[OK] XLY: samples=1185
[OK] XRP-USD: samples=1757
Pooled: (100206, 60, 3) (100206,)
[Fold 1] val_loss=0.6938
[Fold 2] val_loss=0.6884
[Fold 3] val_loss=0.6890
[Fold 4] val_loss=0.6931
[Fold 5] val_loss=0.6850
Saved: /Users/nourf/Projects/stock-predictor/backend/notebooks/models/global/lstm_model.keras /Users/nourf/Projects/stock-predictor/backend/notebooks/models/global/scaler.pkl


In [9]:
def ensure_calibrator(ticker: str, min_points: int = 50, months: int = CAL_WINDOW_MONTHS):
    """Return a dict {"A":..., "B":...}. If file missing, fit now and save."""
    path = os.path.join(CALIB_DIR, f"{ticker.upper()}.json")
    if os.path.exists(path):
        with open(path) as f:
            return json.load(f)

    # Fit on-demand
    end = datetime.utcnow()
    start = end - timedelta(days=int(30*months))
    df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    if df is None or df.empty or len(df) < (LOOK_BACK + HORIZON + 30):
        cal = {"A": 1.0, "B": 0.0, "note": "fallback_identity_insufficient_data"}
        with open(path, "w") as f:
            json.dump(cal, f)
        return cal

    closes = df["Close"].astype(float).values
    scores, y_dir = [], []
    for i in range(LOOK_BACK, len(closes)-HORIZON):
        win = closes[i-LOOK_BACK:i]
        X = build_sequence_features(win)
        X = scaler_all.transform(X)[np.newaxis, ...]
        logit = float(global_model.predict(X, verbose=0).ravel()[0])
        fut = closes[i+HORIZON]/closes[i] - 1.0
        scores.append(logit)
        y_dir.append(1 if fut>0 else 0)

    if len(scores) < min_points:
        cal = {"A": 1.0, "B": 0.0, "note": "fallback_identity_too_few_points"}
        with open(path, "w") as f:
            json.dump(cal, f)
        return cal

    lr = LogisticRegression(max_iter=500).fit(np.array(scores).reshape(-1,1), np.array(y_dir))
    cal = {"A": float(lr.coef_.ravel()[0]), "B": float(lr.intercept_.ravel()[0])}
    with open(path, "w") as f:
        json.dump(cal, f)
    return cal


## 5) Fit Per‑Ticker Calibrators (last N months)

In [10]:

# Reload saved artifacts for clarity
global_model = keras.models.load_model(MODEL_PATH)
scaler_all = joblib.load(SCALER_PATH)

def fit_calibrator_for_ticker(ticker: str, months: int = CAL_WINDOW_MONTHS):
    end = datetime.utcnow()
    start = end - timedelta(days=int(30 * months))
    df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    if df is None or df.empty or len(df) < (LOOK_BACK + HORIZON + 30):
        return None
    closes = df["Close"].astype(float).values

    scores, y_dir = [], []
    for i in range(LOOK_BACK, len(closes) - HORIZON):
        win = closes[i - LOOK_BACK:i]
        X = build_sequence_features(win)             # (L, F)
        X = scaler_all.transform(X)[np.newaxis, ...] # (1, L, F)
        logit = float(global_model.predict(X, verbose=0).ravel()[0])
        fut = closes[i + HORIZON] / closes[i] - 1.0
        scores.append(logit)
        y_dir.append(1 if fut > 0 else 0)

    if len(scores) < 50:
        return None

    lr = LogisticRegression(max_iter=500).fit(np.array(scores).reshape(-1,1), np.array(y_dir))
    A = float(lr.coef_.ravel()[0]); B = float(lr.intercept_.ravel()[0])
    with open(os.path.join(CALIB_DIR, f"{ticker.upper()}.json"), "w") as f:
        json.dump({"A":A, "B":B, "updated_at": datetime.utcnow().isoformat()}, f)
    return {"A":A, "B":B}

ok, skip, err = 0, 0, 0
for t in UNIVERSE:
    try:
        cal = fit_calibrator_for_ticker(t, CAL_WINDOW_MONTHS)
        if cal is None:
            print(f"[SKIP] {t}: insufficient data for calibration")
            skip += 1
        else:
            print(f"[CAL]  {t}: A={cal['A']:.3f} B={cal['B']:.3f}")
            ok += 1
    except Exception as e:
        print(f"[ERR]  {t}: {e}")
        err += 1

print(f"Done. Calibrators OK={ok}, SKIP={skip}, ERR={err}")


[CAL]  AAPL: A=0.871 B=0.040
[CAL]  ABNB: A=1.179 B=-0.106
[CAL]  ADA-USD: A=0.942 B=-0.117
[CAL]  ADBE: A=0.701 B=-0.424
[CAL]  AMD: A=-0.306 B=1.212
[CAL]  AMZN: A=-1.566 B=0.941
[CAL]  ASML: A=0.227 B=0.397
[CAL]  AVAX-USD: A=2.956 B=-0.267
[CAL]  BA: A=-0.352 B=0.872
[CAL]  BKNG: A=0.760 B=0.210
[CAL]  BNB-USD: A=0.608 B=0.263
[CAL]  BRK-B: A=0.573 B=-0.574
[CAL]  BTC-USD: A=0.051 B=0.218
[CAL]  CAT: A=-0.584 B=0.939
[CAL]  COST: A=-0.190 B=0.303
[CAL]  CRM: A=-0.364 B=-0.486
[CAL]  CSCO: A=0.400 B=0.423
[CAL]  DIA: A=0.635 B=0.325
[CAL]  DIS: A=-0.280 B=0.368
[CAL]  DOGE-USD: A=2.669 B=-0.087
[CAL]  DOT-USD: A=2.360 B=-0.419
[CAL]  EEM: A=-0.879 B=1.293
[CAL]  EFA: A=-0.003 B=0.844
[CAL]  ETH-USD: A=0.884 B=0.151
[CAL]  GE: A=0.049 B=0.919
[CAL]  GLD: A=0.096 B=0.502
[CAL]  GOOGL: A=-1.427 B=1.270
[CAL]  HD: A=-0.177 B=0.489
[CAL]  HON: A=1.658 B=-0.189
[CAL]  HYG: A=0.450 B=1.129
[CAL]  INTC: A=-1.139 B=0.467
[CAL]  IWM: A=-1.082 B=1.027
[CAL]  JNJ: A=0.794 B=0.247
[CAL]  JPM: A=


1 Failed download:
['SQ']: YFTzMissingError('possibly delisted; no timezone found')


[SKIP] SQ: insufficient data for calibration
[CAL]  TLT: A=-0.305 B=0.085
[CAL]  TM: A=0.864 B=-0.243
[CAL]  TON-USD: A=1.375 B=-0.154
[CAL]  TSLA: A=0.242 B=0.269
[CAL]  TSM: A=-1.529 B=1.222
[CAL]  UBER: A=-0.732 B=0.410
[CAL]  UNH: A=0.295 B=-0.081
[CAL]  UPS: A=0.452 B=-0.218
[CAL]  V: A=0.970 B=0.103
[CAL]  WMT: A=-0.225 B=0.497
[CAL]  XLB: A=1.399 B=0.246
[CAL]  XLC: A=1.250 B=0.664
[CAL]  XLE: A=-1.179 B=0.530
[CAL]  XLF: A=0.841 B=0.369
[CAL]  XLI: A=1.130 B=0.495
[CAL]  XLK: A=-1.667 B=1.491
[CAL]  XLRE: A=-0.497 B=0.376
[CAL]  XLU: A=0.745 B=0.735
[CAL]  XLV: A=0.052 B=-0.315
[CAL]  XLY: A=-1.253 B=0.922
[CAL]  XRP-USD: A=1.055 B=-0.076
Done. Calibrators OK=78, SKIP=2, ERR=0


## 6) Test Inference — Keep Frontend Shape

In [11]:

def load_calibrator(ticker: str) -> Dict[str,float]:
    path = os.path.join(CALIB_DIR, f"{ticker.upper()}.json")
    if not os.path.exists(path):
        return {"A":1.0, "B":0.0}
    with open(path) as f:
        return json.load(f)

def predict_direction_logit(ticker: str) -> tuple[float, float, pd.Timestamp]:
    end = datetime.utcnow()
    start = end - timedelta(days=int(LOOK_BACK * 3))
    df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    if df is None or df.empty:
        raise RuntimeError(f"No data for {ticker}")
    closes = df["Close"].astype(float).values
    if len(closes) < LOOK_BACK + 1:
        raise RuntimeError("Not enough data for sequence")
    win = closes[-LOOK_BACK:]
    X = build_sequence_features(win)
    X = scaler_all.transform(X)[np.newaxis, ...]
    logit = float(global_model.predict(X, verbose=0).ravel()[0])
    return logit, float(closes[-1]), df.index[-1]

def make_forecast_series(ticker: str, last_close: float, last_date: pd.Timestamp, horizon: int = HORIZON):
    fut_dates = []
    d = last_date.to_pydatetime()
    for _ in range(horizon):
        d = d + timedelta(days=1)
        while d.weekday() >= 5:
            d = d + timedelta(days=1)
        fut_dates.append(d)

    preds = [float(last_close) for _ in range(horizon)]

    start_hist = last_date - pd.Timedelta(days=40)
    df = yf.download(ticker, start=start_hist, end=last_date + pd.Timedelta(days=1),
                     auto_adjust=True, progress=False)

    if df is None or df.empty:
        raise RuntimeError(f"No historical data for {ticker}")

    # Always flatten to a Series with DatetimeIndex
    hist = df["Close"]
    if isinstance(hist, pd.DataFrame):  # happens if yfinance gives multi-index
        hist = hist.squeeze()           # collapse single column DataFrame
    hist.index = pd.to_datetime(hist.index)

    out = []
    for d, v in hist[-20:].items():
        if not isinstance(d, pd.Timestamp):   # if still str, convert
            d = pd.to_datetime(d)
        out.append({
            "date": d.strftime("%Y-%m-%d"),
            "actual": float(v)
        })

    for d, p in zip(fut_dates, preds):
        out.append({
            "date": d.strftime("%Y-%m-%d"),
            "predicted": float(p)
        })

    return out

# ---- Try a sample ticker (change as needed)
TEST_TICKER = "AAPL"

logit, last_close, last_date = predict_direction_logit(TEST_TICKER)
cal = ensure_calibrator(TEST_TICKER)
p_up = float(sigmoid(cal["A"]*logit + cal["B"]))

payload = {
    "ticker": TEST_TICKER,
    "look_back": LOOK_BACK,
    "context": LOOK_BACK,
    "backtest_horizon": HORIZON,
    "horizon": HORIZON,
    "metrics": {"direction_up_prob": p_up},
    "forecast": make_forecast_series(TEST_TICKER, last_close, last_date, HORIZON),
    "recent_backtest": backtest_last_n_days(TEST_TICKER, n_days=30)  # <-- optional, FE can ignore if not used
}


print(json.dumps(payload, indent=2)[:1200] + "\n...")


{
  "ticker": "AAPL",
  "look_back": 60,
  "context": 60,
  "backtest_horizon": 10,
  "horizon": 10,
  "metrics": {
    "direction_up_prob": 0.5688731759178041
  },
  "forecast": [
    {
      "date": "2025-07-29",
      "actual": 211.03050231933594
    },
    {
      "date": "2025-07-30",
      "actual": 208.81301879882812
    },
    {
      "date": "2025-07-31",
      "actual": 207.33470153808594
    },
    {
      "date": "2025-08-01",
      "actual": 202.1505889892578
    },
    {
      "date": "2025-08-04",
      "actual": 203.11949157714844
    },
    {
      "date": "2025-08-05",
      "actual": 202.68995666503906
    },
    {
      "date": "2025-08-06",
      "actual": 213.0082550048828
    },
    {
      "date": "2025-08-07",
      "actual": 219.7805633544922
    },
    {
      "date": "2025-08-08",
      "actual": 229.0900115966797
    },
    {
      "date": "2025-08-11",
      "actual": 227.17999267578125
    },
    {
      "date": "2025-08-12",
      "actual": 229.649993896

In [12]:
def backtest_last_n_days(ticker: str, n_days: int = 30):
    """
    Leak-free rolling backtest for the most recent n_days *trading bars*.
    It expands the download window until it has enough bars.
    """
    # how many bars we need to make n_days forecasts
    required = LOOK_BACK + HORIZON + n_days

    # start with a generous calendar window; expand if needed
    calendar_days = required + 60  # initial buffer
    max_calendar_days = 365 * 2    # hard cap to avoid infinite loops

    df = None
    while True:
        end = datetime.utcnow()
        start = end - pd.Timedelta(days=calendar_days)
        df = yf.download(ticker, start=start, end=end, interval="1d",
                         auto_adjust=True, progress=False)
        if df is not None and not df.empty and len(df) >= required:
            break
        calendar_days = min(int(calendar_days * 1.5), max_calendar_days)
        if calendar_days >= max_calendar_days:
            raise RuntimeError(f"Insufficient data for {ticker} even after expanding window "
                               f"(got {0 if df is None else len(df)} bars, need {required}).")

    # use only the bars we actually downloaded
    df = df.dropna()
    closes = df["Close"].astype(float).values
    dates  = df.index.to_list()

    # ensure we end on the last fully available bar
    last_idx = len(closes) - 1

    # ensure calibrator exists (fits on-demand if missing)
    cal = ensure_calibrator(ticker)

    rows = []
    # we can predict for i in [LOOK_BACK .. last_idx - HORIZON]
    # we want the most recent n_days of those predictions
    start_i = max(LOOK_BACK, last_idx - HORIZON - n_days + 1)
    end_i   = last_idx - HORIZON + 1

    for i in range(start_i, end_i):
        window = closes[i-LOOK_BACK:i]                # only data available up to i
        X = build_sequence_features(window)
        X = scaler_all.transform(X)[np.newaxis, ...]
        logit = float(global_model.predict(X, verbose=0).ravel()[0])
        p_up  = float(sigmoid(cal["A"]*logit + cal["B"]))

        fut_ret = closes[i+HORIZON]/closes[i] - 1.0 if (i+HORIZON) <= last_idx else None
        actual_dir = (1 if fut_ret > 0 else 0) if fut_ret is not None else None

        d = dates[i]
        if not isinstance(d, pd.Timestamp):
            d = pd.to_datetime(d)
        rows.append({
            "date": d.strftime("%Y-%m-%d"),
            "p_up": p_up,
            "logit": logit,
            "actual_return_10d": float(fut_ret) if fut_ret is not None else None,
            "actual_direction": actual_dir,
        })

    return rows


# Example usage:
recent_bt = backtest_last_n_days(TEST_TICKER, n_days=30)
print("Last-30 leak-free backtest rows:", len(recent_bt))
print(recent_bt[:3])


Last-30 leak-free backtest rows: 30
[{'date': '2025-06-30', 'p_up': 0.5299735310963503, 'logit': 0.09152238070964813, 'actual_return_10d': 0.01920365298272375, 'actual_direction': 1}, {'date': '2025-07-01', 'p_up': 0.5357632319865273, 'logit': 0.11822635680437088, 'actual_return_10d': 0.011259788483777067, 'actual_direction': 1}, {'date': '2025-07-02', 'p_up': 0.5435035581974238, 'logit': 0.15399806201457977, 'actual_return_10d': -0.011391427935557585, 'actual_direction': 0}]



## 7) Notes & Tips

- Keep the **`forecast` list shape** identical to your existing API for a drop-in replacement.
- You can expand `metrics` with user-friendly KPIs (directional confidence, typical $ error).
- Schedule this notebook nightly (or convert to a Python script) to refresh the model + calibrators.
- If you need YOLO pattern integration, add a small API that takes a **chart image** and returns pattern detections (stateless) for the UI badges.
